<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/youtube_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask yt-dlp

from flask import Flask, request, render_template_string, send_file, jsonify
import yt_dlp
import os
import threading
import time
import uuid

app = Flask(__name__)

# Папка для временных файлов
TEMP_DIR = "/tmp/youtube_downloads"
if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)

# Словарь для хранения статусов загрузок
download_status = {}

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>YouTube Player</title>
    <style>
        body { font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }
        .container { text-align: center; }
        input { width: 100%; padding: 10px; margin: 10px 0; }
        button { padding: 10px 20px; background: #007bff; color: white; border: none; cursor: pointer; }
        button:hover { background: #0056b3; }
        video { width: 100%; margin-top: 20px; }
        .status { margin: 10px 0; padding: 10px; border-radius: 5px; }
        .loading { background: #fff3cd; color: #856404; }
        .success { background: #d4edda; color: #155724; }
        .error { background: #f8d7da; color: #721c24; }
        #progressBar { width: 100%; height: 20px; margin: 10px 0; }
    </style>
</head>
<body>
    <div class="container">
        <h1>YouTube Video Player</h1>
        <form id="downloadForm">
            <input type="text" id="url" placeholder="Введите ссылку на YouTube видео" required>
            <button type="submit">Скачать и воспроизвести</button>
        </form>

        <div id="status" class="status"></div>

        <div id="playerContainer" style="display: none;">
            <video id="videoPlayer" controls autoplay>
                Ваш браузер не поддерживает видео тег.
            </video>
        </div>
    </div>

    <script>
        document.getElementById('downloadForm').addEventListener('submit', async function(e) {
            e.preventDefault();

            const url = document.getElementById('url').value;
            const statusDiv = document.getElementById('status');
            const playerContainer = document.getElementById('playerContainer');

            // Скрываем предыдущий плеер
            playerContainer.style.display = 'none';

            // Показываем статус загрузки
            statusDiv.textContent = "Начинаем загрузку...";
            statusDiv.className = "status loading";

            try {
                // Отправляем запрос на сервер
                const response = await fetch('/download', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/x-www-form-urlencoded',
                    },
                    body: 'url=' + encodeURIComponent(url)
                });

                const data = await response.json();

                if (data.task_id) {
                    // Начинаем отслеживание прогресса
                    checkDownloadProgress(data.task_id);
                } else if (data.error) {
                    statusDiv.textContent = "Ошибка: " + data.error;
                    statusDiv.className = "status error";
                }
            } catch (error) {
                statusDiv.textContent = "Ошибка соединения с сервером";
                statusDiv.className = "status error";
            }
        });

        function checkDownloadProgress(taskId) {
            const statusDiv = document.getElementById('status');
            const playerContainer = document.getElementById('playerContainer');
            const videoPlayer = document.getElementById('videoPlayer');

            function checkStatus() {
                fetch('/status/' + taskId)
                    .then(response => response.json())
                    .then(data => {
                        if (data.status === 'downloading') {
                            statusDiv.textContent = "Скачивание... " + (data.progress || '');
                            setTimeout(checkStatus, 1000); // Проверяем каждую секунду
                        } else if (data.status === 'completed') {
                            statusDiv.textContent = "Видео готово!";
                            statusDiv.className = "status success";

                            // Показываем плеер и загружаем видео
                            playerContainer.style.display = 'block';
                            videoPlayer.src = '/video/' + data.filename;
                            videoPlayer.load();
                            videoPlayer.play().catch(e => console.log("Автовоспроизведение заблокировано"));

                        } else if (data.status === 'error') {
                            statusDiv.textContent = "Ошибка загрузки: " + data.error;
                            statusDiv.className = "status error";
                        }
                    })
                    .catch(error => {
                        statusDiv.textContent = "Ошибка получения статуса";
                        statusDiv.className = "status error";
                    });
            }

            checkStatus();
        }
    </script>
</body>
</html>
'''

def download_video(task_id, url):
    """Функция для скачивания видео в фоне"""
    try:
        ydl_opts = {
            'outtmpl': os.path.join(TEMP_DIR, f'{task_id}.mp4'),
            'format': 'best[ext=mp4][height<=720]',
            'quiet': False,
            'progress_hooks': [lambda d: update_progress(task_id, d)],
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            download_status[task_id] = {
                'status': 'completed',
                'filename': f'{task_id}.mp4',
                'title': info.get('title', 'video')
            }
    except Exception as e:
        download_status[task_id] = {
            'status': 'error',
            'error': str(e)
        }

def update_progress(task_id, d):
    """Обновление прогресса загрузки"""
    if task_id in download_status:
        if d['status'] == 'downloading':
            percent = d.get('_percent_str', '0%').strip()
            download_status[task_id]['progress'] = percent
        elif d['status'] == 'finished':
            download_status[task_id]['progress'] = '100%'

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/download', methods=['POST'])
def download():
    url = request.form.get('url', '')
    if not url:
        return jsonify({'error': 'URL не указан'})

    # Создаем уникальный ID для задачи
    task_id = str(uuid.uuid4())

    # Инициализируем статус
    download_status[task_id] = {
        'status': 'downloading',
        'progress': '0%'
    }

    # Запускаем загрузку в отдельном потоке
    download_thread = threading.Thread(target=download_video, args=(task_id, url))
    download_thread.daemon = True
    download_thread.start()

    return jsonify({'task_id': task_id})

@app.route('/status/<task_id>')
def get_status(task_id):
    if task_id in download_status:
        return jsonify(download_status[task_id])
    return jsonify({'status': 'not_found'})

@app.route('/video/<filename>')
def serve_video(filename):
    """Отдаем видео файл"""
    filepath = os.path.join(TEMP_DIR, filename)
    if os.path.exists(filepath):
        return send_file(filepath, mimetype='video/mp4')
    return "File not found", 404

# Запускаем сервер
def run_server():
    app.run(host='0.0.0.0', port=5000, debug=False, threaded=True)

# Запускаем сервер в отдельном потоке
server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()

print("Сервер запущен! Ожидаем публичный URL...")

# Получаем публичный URL
from google.colab.output import eval_js
import time

time.sleep(2)
public_url = eval_js("google.colab.kernel.proxyPort(5000)")
print(f"\n✅ Сервер запущен! Откройте эту ссылку в браузере:")
print(f"\n🔗 {public_url}")
print("\n📌 Инструкция:")
print("1. Скопируйте ссылку выше и откройте в новой вкладке браузера")
print("2. Вставьте ссылку на YouTube видео в поле ввода")
print("3. Нажмите 'Скачать и воспроизвести'")
print("4. Видео начнет скачиваться и автоматически появится плеер")

# Оставляем поток активным
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\nСервер остановлен.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.4 MB/s eta 0:00:00
Сервер запущен! Ожидаем публичный URL...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit



✅ Сервер запущен! Откройте эту ссылку в браузере:

🔗 https://5000-m-s-2xaxq0v3372d9-b.europe-west4-2.prod.colab.dev

📌 Инструкция:
1. Скопируйте ссылку выше и откройте в новой вкладке браузера
2. Вставьте ссылку на YouTube видео в поле ввода
3. Нажмите 'Скачать и воспроизвести'
4. Видео начнет скачиваться и автоматически появится плеер


INFO:werkzeug:127.0.0.1 - - [03/Jan/2026 15:14:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2026 15:14:53] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2026 15:15:34] "POST /download HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2026 15:15:35] "GET /status/c2794af2-2a5b-4b54-acea-2e359aa4b682 HTTP/1.1" 200 -


[youtube] Extracting URL: https://www.youtube.com/watch?v=TdSE9hQRjHI
[youtube] TdSE9hQRjHI: Downloading webpage


...

[download]  98.2% of ~ 254.94MiB at    6.58MiB/s ETA 00:01 (frag 360/365)

INFO:werkzeug:127.0.0.1 - - [03/Jan/2026 15:17:23] "GET /status/bb1a1dc8-7cac-4094-a076-356d72388cd0 HTTP/1.1" 200 -


[download] 100% of  252.69MiB in 00:00:29 at 8.50MiB/s                   
[FixupM3u8] Fixing MPEG-TS in MP4 container of "/tmp/youtube_downloads/bb1a1dc8-7cac-4094-a076-356d72388cd0.mp4"


INFO:werkzeug:127.0.0.1 - - [03/Jan/2026 15:17:25] "GET /status/bb1a1dc8-7cac-4094-a076-356d72388cd0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2026 15:17:26] "GET /status/bb1a1dc8-7cac-4094-a076-356d72388cd0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2026 15:17:27] "GET /status/bb1a1dc8-7cac-4094-a076-356d72388cd0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Jan/2026 15:17:27] "GET /video/bb1a1dc8-7cac-4094-a076-356d72388cd0.mp4 HTTP/1.1" 206 -



Сервер остановлен.
